# Coding Assignment - 4

#### Authors:
+ Yogananth Mahalingam - ym24 - MCS-DS
+ Qi Zhou - qizhou8 - MCS-DS
+ Derek Zhang - derekz3 - MCS-DS

#### Contributions:
Yoga, Derek & Qi worked on it independently. Brainstormed together & created the final version.

## Part 2: HMM

In [23]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib.colors import LogNorm



In [24]:
itmax = 100

mx = 3
mz = 2
w = np.array([0.5, 0.5])
A = np.array([[0.5, 0.5], 
              [0.5, 0.5]])
B = np.array([[1/9, 3/9, 5/9], 
              [1/6, 2/6, 3/6]])
# B = np.array([[1/9, 1/6], 
#               [3/9, 2/6], 
#               [5/9, 3/6]])

X = np.loadtxt("coding4_part2_data.txt").astype(int)
print(f"X.shape={X.shape}")
print(f"X={X}")

Z_test = np.genfromtxt("Coding4_part2_Z.txt").astype(int)
print(f"Z_test.shape={Z_test.shape}")
print(f"Z_test={Z_test}")


X.shape=(200,)
X=[2 3 3 3 3 3 3 1 3 3 3 3 3 1 1 3 3 3 2 3 3 3 1 3 1 1 1 2 2 3 3 3 3 3 1 3 3
 2 2 3 3 3 3 1 1 3 3 3 2 3 3 1 3 1 3 3 3 2 1 1 1 1 3 3 3 3 3 3 3 3 1 1 1 3
 3 1 2 1 3 1 1 3 1 1 2 1 3 1 1 2 1 3 3 3 3 1 3 3 3 3 3 3 2 3 3 2 3 2 3 1 3
 3 3 3 1 2 2 3 3 3 3 1 2 2 2 1 3 1 3 1 1 3 1 2 3 3 3 2 2 3 1 3 1 2 1 2 1 3
 1 3 1 3 3 3 1 1 3 2 3 1 3 3 2 3 3 1 1 1 2 3 1 1 2 1 3 3 3 1 2 1 3 2 3 3 3
 3 3 2 1 2 2 3 1 2 3 2 3 2 2 2]
Z_test.shape=(200,)
Z_test=[1 1 1 1 1 1 1 2 1 1 1 1 1 2 2 1 1 1 1 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1 2 1 1
 1 1 1 1 1 1 2 2 1 1 1 1 1 1 2 2 2 1 1 1 1 2 2 2 2 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1
 1 1 1 2 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 1 1 1 2 2 2 2 2 2 1 1 1 1 1 2 2 2 2 2 2 2 2 2 1 1 1 2 2 2 1 1 1 1 1
 1 1 1 2 2 2 2 2 1 1 1 1 1 1 1]


In [25]:
def BW_onstep(X, mx, mz, w, A, B):
    new_w = w
    new_A = A
    new_B = B
    
    alpha = calculate_alpha(X, mx, mz, w, new_A, new_B)
    beta = calculate_beta(X, mx, mz, w, new_A, new_B)
    gamma = calculate_gamma(X, mx, mz, w, A, B, alpha, beta)
    
    new_w = update_w(X, mx, mz, w, A, B, alpha, beta, gamma)
    new_A = update_A(X, mx, mz, w, A, B, alpha, beta, gamma)
    new_B = update_B(X, mx, mz, w, A, B, alpha, beta, gamma)
    
    return (new_w, new_A, new_B)

In [26]:
def calculate_alpha(X, mx, mz, w, A, B):
    n = X.shape[0]
    alpha = np.zeros((n, mz))
    for t in range(n):
        for i in range(mz):
            x = X[t] - 1
            if t == 0:
                #print(f"B[i, :]={B[i, :]}, x={x}")
                alpha[t,i]=w[i] * B[i,x]
            else:
                t_prev = t-1
                #print(f"alpha[t_prev]={alpha[t_prev]}")
                alpha[t,i] = np.sum(alpha[t-1, :] * A[:, i] * B[i, X[t]-1])
    return alpha

# print(w)
# print(A)
# print(B)
# print(B[0, :])
alpha = calculate_alpha(X, mx, mz, w, A, B)
#print(alpha)
    

In [27]:
def calculate_beta(X, mx, mz, w, A, B):
    n = X.shape[0]
    beta = np.ones((n, mz))
    
    for t in reversed(range(n-1)):
        for i in range(mz):
            t_next = t+1
            x = X[t_next]-1

            beta[t,i] = np.sum( A[i, :] * B[:, X[t+1]-1 ] * beta[t+1, :] )
    return beta

#print(B)
beta = calculate_beta(X, mx, mz, w, A, B)
#print(beta)


In [28]:
def calculate_gamma(X, mx, mz, w, A, B, alpha, beta):
    n = X.shape[0]
    gamma = np.zeros((n-1, mz, mz))
    
    for t in range(n-1):
        for i in range(mz):
            for j in range(mz):
                t_next = t+1
                x = X[t_next]-1

                gamma[t, i, j] = alpha[t,i] * A[i,j] * B[j, X[t+1]-1] * beta[t+1, j]
    return gamma

#print(B)
gamma = calculate_gamma(X, mx, mz, w, A, B, alpha, beta)
#print(gamma)
    

In [29]:
def update_w(X, mx, mz, w, A, B, alpha, beta, gamma):
    updated_w = np.sum(gamma[0], axis=1)
    #updated_w = gamma
    #print(gamma[0])
    #print(np.sum(gamma[0], axis=1))
    return updated_w

print(gamma[0:2])
updated_w = update_w(X, mx, mz, w, A, B, alpha, beta, gamma)
print(f"updated_w={updated_w}")

[[[1.90117435e-97 1.71105692e-97]
  [1.90117435e-97 1.71105692e-97]]

 [[2.00123616e-97 1.80111254e-97]
  [1.80111254e-97 1.62100129e-97]]]
updated_w=[3.61223127e-97 3.61223127e-97]


In [30]:
def update_A(X, mx, mz, w, A, B, alpha, beta, gamma):
    updated_A = A
    updated_A = np.sum(gamma, axis=0) / np.sum(np.sum(gamma, axis=2), axis=0)
    return updated_A

updated_A = update_A(X, mx, mz, w, A, B, alpha, beta, gamma)
print(f"updated_A={updated_A}")

updated_A=[[0.485465   0.48503179]
 [0.514535   0.51496821]]


In [31]:
def update_B(X, mx, mz, w, A, B, alpha, beta, gamma):
    updated_B = np.array(B)
#     for i in range(mz):
#         for l in range(mx):
#             x = 1
    return updated_B

updated_B = update_B(X, mx, mz, w, A, B, alpha, beta, gamma)
print(f"updated_B={updated_B}")

updated_B=[[0.11111111 0.33333333 0.55555556]
 [0.16666667 0.33333333 0.5       ]]


In [32]:
def myViterbi(X, mx, mz, w, A, B):
    #Z = np.repeat(1, 200)
    Z = np.array([1])
    return Z

In [33]:
def myBW(X, mx, mz, w, A, B, itmax):
    new_A = A
    new_B = B
    for i in range(itmax):
        (new_w, new_A, new_B) = BW_onstep(X, mx, mz, w, new_A, new_B)
    
    return (new_w, new_A, new_B)
    

In [34]:
(new_w, new_A, new_B) = myBW(X, mx, mz, w, A, B, itmax)
print(f"new_w" )
print(f"{new_w}" )
print(f"new_A" )
print(f"{new_A}" )
print(f"new_B:" )
print(f"{new_B}" )

new_w
[3.21653928e-96 4.57983123e-95]
new_A
[[0.08406424 0.06390481]
 [0.95010617 0.93371112]]
new_B:
[[0.11111111 0.33333333 0.55555556]
 [0.16666667 0.33333333 0.5       ]]


In [35]:
Z = myViterbi(X, mx, mz, w, new_A, new_B)
np.array_equal(Z, Z_test)

False